In [13]:
# ============================
# CHUNK 0: Parámetros & Setup
# ============================
# Requisitos (ejecuta si hace falta):
# !pip install pandas numpy openpyxl xlrd scikit-learn python-dateutil

import os, re, warnings
from typing import Dict, Tuple, List

import numpy as np
import pandas as pd
from IPython.display import display
from dateutil import parser as dtparser
from sklearn.preprocessing import StandardScaler, MinMaxScaler, KBinsDiscretizer
from sklearn.impute import SimpleImputer
from pandas.api.types import (
    is_numeric_dtype, is_datetime64_any_dtype, is_bool_dtype, is_categorical_dtype
)

warnings.filterwarnings("ignore", category=FutureWarning)

# >>>>>>>>>>>> EDITA AQUÍ SI CAMBIA EL NOMBRE DEL ARCHIVO <<<<<<<<<<<<
PATH_DATA_2025 = "/content/DATOS HISTORICOS 2025_TODAS ESTACIONES.xlsx"
PATH_ETIQUETAS = "/content/Etiquetas.xlsx"

# Flags de procesamiento (iguales al flujo 2023)
CREATE_DUMMIES   = True
IMPUTE_STRATEGY  = None        # None | "median" | "mean"
SCALING          = "standard"  # None | "standard" | "minmax"
BINNING          = None        # None | "quantile" | "uniform"
N_BINS           = 5

EXPECTED_COLS = ["PM10","PM2.5","O3","NO","NO2","NOx","SO2","CO","RH","WS","TEMP","SR","BP","WD","RAINF"]
TIME_COL      = "DATETIME"

# Rangos por año (incluye 2025)
RANGOS_POR_ANIO: Dict[int, Dict[str, Tuple[float, float]]] = {
    2020: {"PM10":(0,800),"PM2.5":(0,205.94),"O3":(0,153),"NO":(0,500),"NO2":(0,200),"NOx":(0,500),
           "SO2":(0,200),"CO":(0,20),"RH":(0,100),"WS":(0,75),"TEMP":(0,41),"SR":(0,1),"BP":(690,750),
           "WD":(0,360),"RAINF":(0,30)},
    2021: {"PM10":(0,800),"PM2.5":(0,325),"O3":(0,175),"NO":(0,350),"NO2":(0,100),"NOx":(0,400),
           "SO2":(0,300),"CO":(0,10),"RH":(0,100),"WS":(0,40),"TEMP":(-6.5,45),"SR":(0,1),
           "BP":(690,740),"WD":(0,360),"RAINF":(0,80)},
    2022: {"PM10":(0,999),"PM2.5":(0,450),"O3":(0,160),"NO":(0,400),"NO2":(0,175),"NOx":(0,420),
           "SO2":(0,200),"CO":(0,8),"RH":(0,100),"WS":(0,35),"TEMP":(-5,45),"SR":(0,1.25),
           "BP":(700,740),"WD":(0,360),"RAINF":(0,25)},
    2023: {"PM10":(0,900),"PM2.5":(0,800),"O3":(0,175),"NO":(0,500),"NO2":(0,175),"NOx":(0,500),
           "SO2":(0,250),"CO":(0,14),"RH":(0,100),"WS":(0,40),"TEMP":(0,45),"SR":(0,1),
           "BP":(690,740),"WD":(0,360),"RAINF":(0,70)},
    2024: {"PM10":(0,999),"PM2.5":(0,999),"O3":(0,180),"NO":(0,400),"NO2":(0,130),"NOx":(0,500),
           "SO2":(0,150),"CO":(0,18),"RH":(0,100),"WS":(0,38),"TEMP":(-4,45.5),"SR":(0,1.26),
           "BP":(687.5,740),"WD":(0,360),"RAINF":(0,50)},
    2025: {"PM10":(0,820),"PM2.5":(0,350),"O3":(0,185),"NO":(0,350),"NO2":(0,175),"NOx":(0,400),
           "SO2":(0,405),"CO":(0,10),"RH":(0,100),"WS":(0,40),"TEMP":(-4.5,45),"SR":(0,1.2),
           "BP":(688,740),"WD":(0,360),"RAINF":(0,25)}
}


In [14]:
# ==================================
# CHUNK 1: Utilidades (igual 2023)
# ==================================
def try_parse_datetime(x):
    if pd.isna(x): return pd.NaT
    if isinstance(x, pd.Timestamp): return x
    try:
        return pd.to_datetime(x, errors="coerce")
    except Exception:
        try:
            return pd.to_datetime(dtparser.parse(str(x)), errors="coerce")
        except Exception:
            return pd.NaT

def coerce_numeric(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.replace(r"[^\d\-\.,]", "", regex=True).str.strip()
    s = pd.Series(np.where(s.str.contains(",") & s.str.contains(r"\."), s.str.replace(",", "", regex=False), s))
    s = pd.Series(np.where((~pd.isna(s)) & (pd.Series(s).str.contains(",") & ~pd.Series(s).str.contains(r"\.")),
                 pd.Series(s).str.replace(".", "", regex=False).str.replace(",", ".", regex=False),
                 s))
    return pd.to_numeric(pd.Series(s), errors="coerce")

def standardize_column_names(cols: List[str]) -> List[str]:
    mapping = {
        r"^pm\s*10$":"PM10", r"^pm10$":"PM10",
        r"^pm\s*2\.?5$":"PM2.5", r"^pm2\.?5$":"PM2.5",
        r"^o3$":"O3", r"^no$":"NO", r"^no2$":"NO2", r"^nox$":"NOx",
        r"^so2$":"SO2", r"^co$":"CO", r"^rh$":"RH", r"^ws$":"WS",
        r"^(temp|temperature)$":"TEMP", r"^sr$":"SR", r"^bp$":"BP",
        r"^wd$":"WD", r"^(rain|rainf|rainfall|precip.*)$":"RAINF",
        r"^(date|datetime|fecha.*hora|fecha_hora|time.*)$":TIME_COL,
        r"^(estacion|estación|station)$":"STATION",
    }
    new_cols = []
    for c in cols:
        c2 = re.sub(r"\s+", " ", str(c).strip()).upper()
        new = c2
        for pat, tgt in mapping.items():
            if re.match(pat, c2, flags=re.IGNORECASE):
                new = tgt; break
        new_cols.append(new)
    return new_cols

def add_station_if_missing(df: pd.DataFrame, station_name: str) -> pd.DataFrame:
    if "STATION" not in df.columns:
        df["STATION"] = station_name
    return df

def concat_sheets_with_station(xls: pd.ExcelFile, sheet_names: List[str]) -> pd.DataFrame:
    dfs = []
    for sh in sheet_names:
        try:
            tmp = xls.parse(sh)
            tmp.columns = standardize_column_names(list(tmp.columns))
            tmp = add_station_if_missing(tmp, sh)
            dfs.append(tmp)
        except KeyError as e:
            print(f"Error processing sheet '{sh}': {e}")
            # Optionally, you can choose to skip the problematic sheet or handle the error differently
            continue
    return pd.concat(dfs, ignore_index=True)

In [15]:
# =====================================================
# CHUNK 2: 2025 → Crear TABLA MAESTRA desde 15 sheets
# =====================================================
assert os.path.exists(PATH_DATA_2025), f"No se encontró {PATH_DATA_2025}"

xls_2025 = pd.ExcelFile(PATH_DATA_2025)
df_master_2025 = concat_sheets_with_station(xls_2025, xls_2025.sheet_names)

# Normaliza nombres, asegura columnas esperadas y parsea DATETIME
df_master_2025.columns = standardize_column_names(df_master_2025.columns.tolist())

# Keep only expected columns plus TIME_COL and STATION
cols_to_keep = EXPECTED_COLS + [TIME_COL, "STATION"]
df_master_2025 = df_master_2025[[c for c in cols_to_keep if c in df_master_2025.columns]]

for c in EXPECTED_COLS:
    if c not in df_master_2025.columns:
        df_master_2025[c] = np.nan

if TIME_COL in df_master_2025.columns:
    df_master_2025[TIME_COL] = df_master_2025[TIME_COL].apply(try_parse_datetime)

# Apply coerce_numeric to expected columns before saving
for c in EXPECTED_COLS:
    if c in df_master_2025.columns:
        df_master_2025[c] = coerce_numeric(df_master_2025[c])


print("Dimensión tabla maestra 2025:", df_master_2025.shape)
print("Estaciones detectadas:", sorted(df_master_2025["STATION"].dropna().unique().tolist())[:20], "...")

# Guarda la tabla maestra por si quieres usarla después directamente
os.makedirs("/content/master", exist_ok=True)
df_master_2025.to_parquet("/content/master/aq_2025_master.parquet", index=False)
df_master_2025.to_csv("/content/master/aq_2025_master.csv", index=False)
print("✅ Master 2025 exportada en /content/master")

Dimensión tabla maestra 2025: (64974, 17)
Estaciones detectadas: ['CE', 'NE', 'NE2', 'NE3', 'NO', 'NO2', 'NO3', 'NTE', 'NTE2', 'SE', 'SE2', 'SE3', 'SO', 'SO2', 'SUR'] ...
✅ Master 2025 exportada en /content/master


In [16]:
# ============================================
# CHUNK 3: Funciones del pipeline (igual 2023)
# ============================================
def describe_variables(df: pd.DataFrame, etiquetas: pd.DataFrame | None) -> pd.DataFrame:
    summary = []
    for c in df.columns:
        ser = df[c]; n_null = int(ser.isna().sum())
        if is_datetime64_any_dtype(ser):
            tipo = "Fecha/Hora"
            vals = [pd.to_datetime(ser.min()).isoformat(), pd.to_datetime(ser.max()).isoformat()] if ser.notna().any() else [np.nan, np.nan]
        elif is_bool_dtype(ser):
            tipo = "Categórico (bool)"; uniq = ser.dropna().unique().tolist(); vals = uniq[:20] + (["..."] if len(uniq) > 20 else [])
        elif is_categorical_dtype(ser) or ser.dtype == "object":
            tipo = "Categórico"; uniq = ser.dropna().unique().tolist(); vals = uniq[:20] + (["..."] if len(uniq) > 20 else [])
        elif is_numeric_dtype(ser):
            tipo = "Numérico"
            if ser.notna().any():
                vmin, vmax = np.nanmin(ser.astype(float)), np.nanmax(ser.astype(float))
                vals = [float(vmin), float(vmax)]
            else:
                vals = [np.nan, np.nan]
        else:
            tipo = str(ser.dtype); uniq = ser.dropna().unique().tolist(); vals = uniq[:20] + (["..."] if len(uniq) > 20 else [])
        summary.append({"variable": c, "descripcion": None, "tipo": tipo, "valores_posibles": vals, "nulos": n_null})
    desc_df = pd.DataFrame(summary)

    if etiquetas is not None:
        et = etiquetas.copy()
        if "variable" not in et.columns:
            cand = [col for col in et.columns if "var" in col.lower() or "nombre" in col.lower()]
            if cand: et = et.rename(columns={cand[0]: "variable"})
        for src, dst in [("descripcion","descripcion"),("descripción","descripcion"),("tipo","tipo_usuario"),
                         ("valores","valores_usuario"),("valores_posibles","valores_usuario")]:
            if src in et.columns and dst not in et.columns: et = et.rename(columns={src: dst})
        if "variable" in et.columns:
            desc_df = desc_df.merge(et, on="variable", how="left")
            if "tipo_usuario" in desc_df.columns: desc_df["tipo"] = desc_df["tipo_usuario"].fillna(desc_df["tipo"])
            if "valores_usuario" in desc_df.columns: desc_df["valores_posibles"] = desc_df["valores_usuario"].combine_first(desc_df["valores_posibles"])
            for c in ["tipo_usuario","valores_usuario"]:
                if c in desc_df.columns: desc_df.drop(columns=[c], inplace=True)
    return desc_df

def quality_checks(df: pd.DataFrame) -> dict:
    return {
        "shape": df.shape,
        "missing_by_col": df.isna().sum().sort_values(ascending=False).to_dict(),
        "has_infinite": np.isinf(df.select_dtypes(include=[np.number]).to_numpy()).any(),
        "duplicated_rows": int(df.duplicated().sum())
    }

def select_and_explain(df: pd.DataFrame):
    keep = [TIME_COL, "STATION"] + EXPECTED_COLS
    keep = [c for c in keep if c in df.columns]
    out = df[keep].copy()
    msg = ("Se seleccionaron DATETIME, STATION y variables estándar; "
           "se excluyeron columnas auxiliares para evitar ruido.")
    return out, msg

def identify_targets(df: pd.DataFrame): return [c for c in ["PM2.5","PM10"] if c in df.columns]

def fix_types(df: pd.DataFrame) -> pd.DataFrame:
    # Apply coerce_numeric only to columns in EXPECTED_COLS that exist in the DataFrame
    for c in EXPECTED_COLS:
        if c in df.columns:
            df[c] = coerce_numeric(df[c])

    if TIME_COL in df.columns:
        if not is_datetime64_any_dtype(df[TIME_COL]): df[TIME_COL] = pd.to_datetime(df[TIME_COL], errors="coerce")
        df["YEAR"]  = df[TIME_COL].dt.year
        df["MONTH"] = df[TIME_COL].dt.month
        df["DAY"]   = df[TIME_COL].dt.day
        df["HOUR"]  = df[TIME_COL].dt.hour
    return df


def apply_yearly_ranges(df: pd.DataFrame) -> pd.DataFrame:
    if "YEAR" not in df.columns: return df
    def clip_row(row):
        yr = row["YEAR"]; ranges = RANGOS_POR_ANIO.get(int(yr) if not pd.isna(yr) else yr, {})
        for col, (lo, hi) in ranges.items():
            if col in row and pd.notna(row[col]) and (row[col] < lo or row[col] > hi): row[col] = np.nan
        return row
    return df.apply(clip_row, axis=1)

def drop_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    b=len(df); df=df.drop_duplicates(); a=len(df); print(f"Duplicados (exactos): {b-a}")
    if TIME_COL in df.columns and "STATION" in df.columns:
        b=len(df); df=df.sort_values(TIME_COL).drop_duplicates(subset=["STATION", TIME_COL], keep="first"); a=len(df)
        print(f"Duplicados (STATION, DATETIME): {b-a}")
    return df

def iqr_outlier_mask(x: pd.Series, k: float = 1.5):
    q1, q3 = np.nanpercentile(x, [25, 75]); iqr = q3-q1; lo, hi = q1-k*iqr, q3+k*iqr
    return (x < lo) | (x > hi)

def handle_outliers(df: pd.DataFrame, k: float = 1.5) -> pd.DataFrame:
    if "STATION" not in df.columns or "YEAR" not in df.columns: return df
    num_cols = [c for c in EXPECTED_COLS if c in df.columns]
    def _proc(g):
        for c in num_cols:
            s=g[c]
            if s.notna().sum()>=12:
                g.loc[iqr_outlier_mask(s,k), c]=np.nan
        return g
    return df.groupby(["STATION","YEAR"], group_keys=False).apply(_proc)

def handle_missing(df: pd.DataFrame) -> pd.DataFrame:
    num_cols = [c for c in df.columns if c in EXPECTED_COLS]
    if TIME_COL in df.columns and "STATION" in df.columns:
        def _interp(g):
            g=g.sort_values(TIME_COL).set_index(TIME_COL)
            for c in num_cols:
                s=g[c].interpolate(method="time").ffill().bfill()
                g[c]=s
            return g.reset_index()
        df=df.groupby("STATION", group_keys=False).apply(_interp)
    if IMPUTE_STRATEGY in ("mean","median"):
        imputer=SimpleImputer(strategy=IMPUTE_STRATEGY)
        df[num_cols]=imputer.fit_transform(df[num_cols])
    return df

def handle_categoricals(df: pd.DataFrame) -> pd.DataFrame:
    cat_cols=[c for c in df.columns if df[c].dtype=="object" or str(df[c].dtype).startswith("category")]
    return pd.get_dummies(df, columns=cat_cols, drop_first=True, dtype=int) if CREATE_DUMMIES and cat_cols else df

def build_features(df: pd.DataFrame) -> pd.DataFrame:
    out=df.copy()
    if "WS" in out.columns and "WD" in out.columns:
        ws=out["WS"].astype(float); wd=np.deg2rad(out["WD"].astype(float))
        out["WIND_U"]=-ws*np.sin(wd); out["WIND_V"]=-ws*np.cos(wd)
    if TIME_COL in out.columns:
        out["WEEKDAY"]=out[TIME_COL].dt.weekday; out["IS_WEEKEND"]=(out["WEEKDAY"]>=5).astype(int)
    return out

def scale_numeric(df: pd.DataFrame):
    if SCALING not in ("standard","minmax"): return df, {}
    cols=[c for c in (EXPECTED_COLS+["WIND_U","WIND_V"]) if c in df.columns]
    scaler=StandardScaler() if SCALING=="standard" else MinMaxScaler()
    out=df.copy(); out[cols]=scaler.fit_transform(out[cols]); return out, {"scaler":SCALING,"columns":cols}

def bin_numeric(df: pd.DataFrame, cols: List[str]):
    if BINNING not in ("quantile","uniform") or not cols: return df, {}
    b=KBinsDiscretizer(n_bins=N_BINS, encode="ordinal", strategy=("quantile" if BINNING=="quantile" else "uniform"))
    x=df[cols]; mask=x.notna().all(axis=1); out=df.copy()
    out.loc[mask,[f"{c}_BIN" for c in cols]]=b.fit_transform(x[mask])
    return out, {"binner":BINNING,"columns":cols,"bins":N_BINS}

def tidy_and_export(df: pd.DataFrame, outdir="/content/clean_2025"):
    os.makedirs(outdir, exist_ok=True)
    df.to_parquet(os.path.join(outdir,"air_quality_2025_clean_full.parquet"), index=False)
    if "YEAR" in df.columns:
        for yr, dfg in df.groupby("YEAR"):
            dfg.to_csv(os.path.join(outdir, f"air_quality_2025_clean_{int(yr)}.csv"), index=False)
    if "STATION" in df.columns and "YEAR" in df.columns:
        for (st, yr), dfg in df.groupby(["STATION","YEAR"]):
            fname=re.sub(r"[^A-Za-z0-9\-]+","_", str(st))
            dfg.to_csv(os.path.join(outdir, f"clean_{fname}_{int(yr)}.csv"), index=False)
    print("📦 Exportado en:", outdir)

In [17]:
# =======================================
# CHUNK 4: Ejecutar pipeline sobre 2025
# =======================================
print("=== 2025: Comprensión de los datos ===")
etiquetas = pd.read_excel(PATH_ETIQUETAS) if os.path.exists(PATH_ETIQUETAS) else None
print("Dimensión maestra 2025:", df_master_2025.shape)

desc = describe_variables(df_master_2025, etiquetas)
display(desc.head(30))

qc = quality_checks(df_master_2025)
print("Calidad:")
print(" - Registros/Columnas:", qc["shape"])
print(" - Faltantes (top10):", dict(list(qc["missing_by_col"].items())[:10]))
print(" - Infinitos:", qc["has_infinite"])
print(" - Duplicados:", qc["duplicated_rows"])

print("\n=== 2025: Preparación ===")
df, rationale = select_and_explain(df_master_2025)
print("Racional:\n", rationale)

df = fix_types(df)
df = apply_yearly_ranges(df)      # aplica rangos del año 2025
df = drop_duplicates(df)
df = handle_outliers(df, k=1.5)
df = handle_missing(df)

targets = identify_targets(df)
print("Targets sugeridos:", targets)

df = handle_categoricals(df)

print("\n=== 2025: Transformación ===")
df = build_features(df)
df_scaled, scale_info = scale_numeric(df)
if scale_info: print("Escalado:", scale_info)

bin_cols = [c for c in ["PM2.5","PM10"] if c in df_scaled.columns]
df_final, bin_info = bin_numeric(df_scaled, bin_cols)
if bin_info: print("Binning:", bin_info)

print("\n=== 2025: Reformateo / Exportación ===")
tidy_and_export(df_final)

print("\n=== Resumen 2025 ===")
print("Shape final:", df_final.shape)
print("NA totales:", int(df_final.isna().sum().sum()))
print("Features derivadas:", [c for c in ["WIND_U","WIND_V","WEEKDAY","IS_WEEKEND"] if c in df_final.columns])
print("✅ Listo para análisis como la versión 2023.")


=== 2025: Comprensión de los datos ===
Dimensión maestra 2025: (64974, 17)


/tmp/ipython-input-1214294706.py:13: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif is_categorical_dtype(ser) or ser.dtype == "object":
/tmp/ipython-input-1214294706.py:13: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif is_categorical_dtype(ser) or ser.dtype == "object":
/tmp/ipython-input-1214294706.py:13: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif is_categorical_dtype(ser) or ser.dtype == "object":
/tmp/ipython-input-1214294706.py:13: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif is_categorical_dtype(ser) or ser.dtype == "object":
/tmp/ipython-input-121429470

,variable,descripcion,tipo,valores_posibles,nulos
0,PM10,None,Numérico,"[2.0, 811.0]",2537
1,PM2.5,None,Numérico,"[0.0, 340.25]",24036
2,O3,None,Numérico,"[0.5, 184.0]",3608
3,NO,None,Numérico,"[0.5, 325.6]",13847
4,NO2,None,Numérico,"[0.1, 165.1]",12640
5,NOx,None,Numérico,"[0.5, 399.9]",11600
6,SO2,None,Numérico,"[0.5, 404.7]",6945
7,CO,None,Numérico,"[0.05, 9.79]",8218
8,RH,None,Numérico,"[0.0, 100.0]",9471
9,SR,None,Numérico,"[-0.008, 2.0]",3888


Calidad:
 - Registros/Columnas: (64974, 17)
 - Faltantes (top10): {'WS': 64974, 'BP': 64974, 'TEMP': 64974, 'WD': 64974, 'PM2.5': 24036, 'NO': 13847, 'NO2': 12640, 'NOx': 11600, 'RH': 9471, 'CO': 8218}
 - Infinitos: False
 - Duplicados: 0

=== 2025: Preparación ===
Racional:
 Se seleccionaron DATETIME, STATION y variables estándar; se excluyeron columnas auxiliares para evitar ruido.
Duplicados (exactos): 0
Duplicados (STATION, DATETIME): 0


/tmp/ipython-input-1214294706.py:105: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby(["STATION","YEAR"], group_keys=False).apply(_proc)
/tmp/ipython-input-1214294706.py:116: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df=df.groupby("STATION", group_keys=False).apply(_interp)
/usr/local/lib/python3.12/dist-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in

Targets sugeridos: ['PM2.5', 'PM10']

=== 2025: Transformación ===
Escalado: {'scaler': 'standard', 'columns': ['PM10', 'PM2.5', 'O3', 'NO', 'NO2', 'NOx', 'SO2', 'CO', 'RH', 'WS', 'TEMP', 'SR', 'BP', 'WD', 'RAINF', 'WIND_U', 'WIND_V']}

=== 2025: Reformateo / Exportación ===
📦 Exportado en: /content/clean_2025

=== Resumen 2025 ===
Shape final: (64959, 38)
NA totales: 407034
Features derivadas: ['WIND_U', 'WIND_V', 'WEEKDAY', 'IS_WEEKEND']
✅ Listo para análisis como la versión 2023.
